In [21]:
import yaml
import re

with open('mattermost-openapi-v4.yaml') as file:
    api_data = yaml.load(file)['paths']
    api_functions = []

    for path in api_data:
        if "post" in api_data[path]:
            api_functions.append({
                "description": api_data[path]["post"]["description"].split("#####")[0].replace('\n', '\n\t\t\t').rstrip(),
                "path": path,
                "method": "post"
            })

        if "put" in api_data[path]:
            api_functions.append({
                "description": api_data[path]["put"]["description"].split("#####")[0].replace('\n', '\n\t\t\t').rstrip(),
                "path": path,
                "method": "put"})
            
with open('mrequests.py') as file:
    implemented = {"get":[], "post":[], "put": [], "delete":[]}
    for line in file:
        if re.search("self\.send_request\(.*\)", line):
            path = line.split("'")[1].strip()
            method = line.split("'")[3].strip()
            implemented[method].append(path)
    

with open('./mattermost_api_post_put.py', "w") as file:
    for function in api_functions:
        #function_name="FUNCTION_NAME" #input()
        if function["path"] not in implemented[function["method"]]:
            #print( function["path"], function["method"])
            function_schema = '    def FUNCTION_NAME(self, **kwargs):\n        """{0}"""\n        return self.send_request(\'{1}\', \'{2}\', **kwargs)\n\n'.format(function["description"], function["path"], function["method"])
            file.write(function_schema)
